## Import Data

In [48]:
import pandas as pd
import numpy as np

In [49]:
# use pandas to load data into a DataFrame
# df = pd.read_csv("/home/danielbudi/Collage/comp-gammafest/comp-model/comp-dataset/mapped-v2_imputed_mf5iter.csv")
# df = pd.read_csv("/home/danielbudi/Collage/comp-gammafest/comp-model/comp-dataset/encode data/train.csv")
df = pd.read_csv("imputed_mf10iter_drop_outliers.csv")
df.shape # (rows, columns)

(32629, 39)

In [50]:
TARGET_COLUMN = 'DC201'
df_sampling = df.copy()

In [51]:
FEATURE_LIST = list(df.columns)
NUMERICAL_DATA = ['DC216', 'DC220', 'DC142a']
CATEGORICAL_DATA = [column for column in df.columns if column != TARGET_COLUMN and column not in NUMERICAL_DATA]

COLUMN_CATEGORICAL_INDEX = []

for column in CATEGORICAL_DATA:
    COLUMN_CATEGORICAL_INDEX.append(df.columns.get_loc(column))

## Pre-Processing Data

### Split Feature and Labels

In [52]:
label_df = df[TARGET_COLUMN]
# label = pd.DataFrame(label)
df = df.drop(TARGET_COLUMN, axis=1)
feature_df = df

label_sampling = df_sampling[TARGET_COLUMN]
# label = pd.DataFrame(label)
df_sampling = df_sampling.drop(TARGET_COLUMN, axis=1)
feature_sampling = df_sampling

### SMOTE

In [53]:
# from collections import Counter
# from imblearn.over_sampling import SMOTENC

# counter = Counter(label_sampling)
# print(counter)

# oversample = SMOTENC(sampling_strategy=0.2,
#                      categorical_features=COLUMN_CATEGORICAL_INDEX,
#                      random_state=42)
# feature_sampling, label_sampling = oversample.fit_resample(feature_sampling, label_sampling)

# counter = Counter(label_sampling)
# print(counter)

### Feature Importance

#### Non-SMOTE

In [54]:
# import matplotlib.pyplot as plt
# from xgboost import XGBClassifier
# from xgboost import plot_importance

# # fit model no training data
# model = XGBClassifier()
# model.fit(feature_df, label_df)
# # feature importance
# print(model.feature_importances_)
# # plot
# fig, ax = plt.subplots(figsize=(10,10))
# plot_importance(model, ax=ax)
# plt.show()

In [55]:
# np.sort(feature_df['DC205'].unique())

In [56]:
# feature_df.nunique()

In [57]:
# feature_df_importance = feature_df[['DC024', 'DC142a', 'DC220', 'DC214', 'DC213', 'DC216',
#                                     'DC205', 'DC235', 'DC270a', 'DC252', 'DC215', 'DC226', 'DC217']]
# feature_df_importance

#### SMOTE

In [58]:
# # fit model no training data
# model = XGBClassifier()
# model.fit(feature_sampling, label_sampling)
# # feature importance
# print(model.feature_importances_)
# # plot
# fig, ax = plt.subplots(figsize=(10,10))
# plot_importance(model, ax=ax)
# plt.show()

In [59]:
# np.sort(feature_sampling['DC205'].unique())

In [60]:
# feature_sampling.nunique()

In [61]:
# feature_sampling_importance = feature_sampling[['DC024', 'DC142a', 'DC220', 'DC214', 'DC213', 'DC216', 'DC109',
#                                                 'DC205', 'DC235', 'DC270a', 'DC252', 'DC215', 'DC226', 'DC230a']]
# feature_sampling_importance

In [62]:
test_df = pd.read_csv('../datasets/test.csv')
test_id = test_df.pop('id')

### Normalization

In [63]:
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
# scaler = StandardScaler()

# train set
numerical_df_train = feature_sampling[NUMERICAL_DATA].astype(float).copy()
categorical_df_train = feature_sampling[CATEGORICAL_DATA].astype('category').copy()

# normalization
scaler.fit(numerical_df_train)
numerical_df_train = scaler.transform(numerical_df_train)
numerical_df_train = pd.DataFrame(numerical_df_train, columns=NUMERICAL_DATA)


# test set
numerical_df_test = test_df[NUMERICAL_DATA].astype(float).copy()
categorical_df_test = test_df[CATEGORICAL_DATA].astype('category').copy()

# normalization
numerical_df_test = scaler.transform(numerical_df_test)
numerical_df_test = pd.DataFrame(numerical_df_test, columns=NUMERICAL_DATA)

# Create an instance of the OneHotEncoder
encoder = ce.OneHotEncoder(cols=CATEGORICAL_DATA, use_cat_names=True)

# Fit the encoder on the training data
encoder.fit(categorical_df_train)

one_hot_df_train = encoder.transform(categorical_df_train)
one_hot_df_test = encoder.transform(categorical_df_test)

merged_df_train = pd.concat([numerical_df_train, one_hot_df_train], axis=1)
merged_df_test = pd.concat([numerical_df_test, one_hot_df_test], axis=1)

In [64]:
# merged_df_train
# merged_df_test

## Modelling

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

TEST_SIZE = 0.2
RANDOM_SEED = 42
FOLD = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

# X_train, X_test, y_train, y_test = train_test_split(X_train_norm, label_sampling, test_size=TEST_SIZE, random_state=RANDOM_SEED) # All Normalized
X_train, X_test, y_train, y_test = train_test_split(merged_df_train, label_sampling, test_size = TEST_SIZE, random_state = RANDOM_SEED) # Numeric Normalized
# X_train, X_test, y_train, y_test = train_test_split(feature_sampling, label_sampling, test_size=TEST_SIZE, random_state=RANDOM_SEED) # No Normalized
# X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df, test_size=TEST_SIZE, random_state=RANDOM_SEED) # No Normalized
# X_train, X_test, y_train, y_test = feature_sampling, feature_df, label_sampling, label_df

### Hyperparameter Tuning

In [66]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_score

In [67]:
# defining parameter range
# param_grid = {'learning_rate': [0.1, 0.01, 0.001],
#               'min_child_weight':[1, 2, 3],
#               'gamma':[0, 0.1, 0.001]}

# grid = GridSearchCV(estimator=xgb,
#                     param_grid=param_grid,
#                     scoring='f1_micro', n_jobs=-1,
#                     cv=FOLD, verbose=3)
  
# fitting the model for grid search
# grid.fit(X_train, y_train) # Split the training and validation
# grid.fit(X_train_norm, label) # All the training with normalized
# grid.fit(X_resample, label) # All the training with numeric normalized
# grid.fit(feature, label) # All the features no normalized

# print(grid.best_params_)

# grid_predictions = grid.best_estimator_.predict(X_test)
# print(classification_report(y_test, grid_predictions, digits=5))

In [68]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

# Create a LightGBM dataset
data = lgb.Dataset(merged_df_train, label=label_sampling)

# Set the parameters for the LightGBM model
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting_type": "dart",
    "n_estimators": 1000, 
    "subsample": 0.8,
    # "olsample_bytree": 0.8,
    "scale_pos_weight":2, 
    "num_leaves": 100,
    "random_state": 42, 
    "learning_rate": 0.1, 
    "min_child_weight":2, 
    "max_depth": 22
}

# Set the number of folds for cross-validation
n_folds = 5

# Initialize a KFold object
kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)

# Initialize a list to store the F1 score of each fold
f1_list = []

# Perform cross-validation
for train_index, test_index in kf.split(merged_df_train):
    # Split the data into training and test sets
    X_train, X_test = merged_df_train.iloc[train_index], merged_df_train.iloc[test_index]
    y_train, y_test = label_sampling.iloc[train_index], label_sampling.iloc[test_index]

    # Create a LightGBM dataset for the training data
    train_data = lgb.Dataset(X_train, label=y_train)

    # Train the model
    model = lgb.train(params, train_data)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Convert the predicted probabilities to binary labels
    y_pred = [round(x) for x in y_pred]

    # Calculate the F1 score for this fold
    f1 = f1_score(y_test, y_pred)
    f1_list.append(f1)

# Calculate the average F1 score across all folds
mean_f1 = sum(f1_list) / n_folds

print(f"Average F1 score: {mean_f1:.4f}")

e:\Projects\comp-gammafest\venv\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 23438, number of negative: 2665
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 549
[LightGBM] [Info] Number of data points in the train set: 26103, number of used features: 156
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.897904 -> initscore=2.174155
[LightGBM] [Info] Start training from score 2.174155


e:\Projects\comp-gammafest\venv\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 23404, number of negative: 2699
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002662 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 548
[LightGBM] [Info] Number of data points in the train set: 26103, number of used features: 156
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.896602 -> initscore=2.160026
[LightGBM] [Info] Start training from score 2.160026


e:\Projects\comp-gammafest\venv\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 23374, number of negative: 2729
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002279 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 543
[LightGBM] [Info] Number of data points in the train set: 26103, number of used features: 155
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.895453 -> initscore=2.147689
[LightGBM] [Info] Start training from score 2.147689


e:\Projects\comp-gammafest\venv\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 23432, number of negative: 2671
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 547
[LightGBM] [Info] Number of data points in the train set: 26103, number of used features: 156
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.897675 -> initscore=2.171650
[LightGBM] [Info] Start training from score 2.171650


e:\Projects\comp-gammafest\venv\Lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Number of positive: 23416, number of negative: 2688
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002506 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 547
[LightGBM] [Info] Number of data points in the train set: 26104, number of used features: 155
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.897027 -> initscore=2.164622
[LightGBM] [Info] Start training from score 2.164622
Average F1 score: 0.9519


In [69]:
prediction = model.predict(X_test)
result = []

for pred in prediction:
  result.append(1 if pred >= 0.5 else 0)
print(classification_report(y_test, result, digits=5))

              precision    recall  f1-score   support

           0    0.72065   0.26370   0.38612       675
           1    0.92083   0.98821   0.95333      5850

    accuracy                        0.91326      6525
   macro avg    0.82074   0.62595   0.66972      6525
weighted avg    0.90013   0.91326   0.89465      6525



In [70]:
lgbm = LGBMClassifier(boosting_type='dart', n_estimators=1000, subsample=0.8,
                      colsample_bytree=0.8, scale_pos_weight=2, num_leaves=100,
                      random_state=42, learning_rate=0.1, min_child_weight=2, max_depth=22)
lgbm.fit(X_train, y_train)

prediction = lgbm.predict(X_test)
print(classification_report(y_test, prediction, digits=5))

              precision    recall  f1-score   support

           0    0.70916   0.26370   0.38445       675
           1    0.92078   0.98752   0.95299      5850

    accuracy                        0.91264      6525
   macro avg    0.81497   0.62561   0.66872      6525
weighted avg    0.89889   0.91264   0.89417      6525



In [71]:
from sklearn.metrics import f1_score

# Make probability predictions on the validation data
y_prob = lgbm.predict_proba(X_test)[:, 1]

# Compute the F1 score for different threshold values
thresholds = np.arange(0, 1.01, 0.01)
f1_scores = [f1_score(y_test, y_prob > t) for t in thresholds]

# Find the threshold that gives the highest F1 score
best_threshold = thresholds[np.argmax(f1_scores)]
best_f1_score = np.max(f1_scores)

print(f'Best threshold: {best_threshold}')
print(f'Best F1 score: {best_f1_score}')

# Make binary predictions using the best decision threshold
y_pred = y_prob > best_threshold

print(classification_report(y_test,y_pred, digits=4))

Best threshold: 0.5
Best F1 score: 0.9529858132629495
              precision    recall  f1-score   support

           0     0.7092    0.2637    0.3844       675
           1     0.9208    0.9875    0.9530      5850

    accuracy                         0.9126      6525
   macro avg     0.8150    0.6256    0.6687      6525
weighted avg     0.8989    0.9126    0.8942      6525



## Prediction

### Predict

In [72]:
# grid_predictions = grid.best_estimator_.predict(test_norm)
# grid_predictions = grid.best_estimator_.predict(test_df1)
# grid_predictions = xgb.predict(test_norm)
grid_predictions = lgbm.predict(merged_df_test)
# grid_predictions = model.predict(test_copy)
grid_predictions[-100:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [73]:
# Custom Threshold

# P_prod_submit = lgbm.predict_proba(test_norm)[:, 1] > best_threshold
# P_prod_submit = grid.best_estimator_.predict(X_test_submit)
# P_prod_submit

In [74]:
result = []
for res in grid_predictions:
# for res in P_prod_submit:
  result.append('Layak Minum' if res==1 else 'Tidak Layak Minum')

finish_pd = pd.DataFrame({'id':test_id.values, 'DC201':result})
finish_pd

,id,DC201
0,26718,Layak Minum
1,26802,Layak Minum
2,41302,Layak Minum
3,38698,Layak Minum
4,44257,Layak Minum
...,...,...
11985,36943,Layak Minum
11986,33415,Layak Minum
11987,41998,Layak Minum
11988,41567,Layak Minum


In [75]:
finish_pd['DC201'].value_counts()

DC201
Layak Minum          11546
Tidak Layak Minum      444
Name: count, dtype: int64

In [76]:
# finish_pd.to_csv('/home/danielbudi/Collage/comp-gammafest/comp-model/comp-dataset/result-file/result-LGBM-one_hot_encode-no_SMOTE-MinMax-Cat_modus_num_imputation.csv', index=False)